<a href="https://colab.research.google.com/github/Francho03/Praca-licencjacka/blob/main/codes/Agregowanie_danych_do_powiatow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np


file_path = "/content/drive/MyDrive/praca_licencjacka/Sprawozdanie_przemoc_2014_2023.xlsx"
columns = ["lp", "woj", "pow", "gm", "typ_gm", "typ_jednostki", "keso", "rok",
           "liczba_proc_nk_a", "liczba_proc_nk_c", "rodziny_wszczete",
           "liczba_osob_ogol", "liczba_osob_kobiety", "liczba_osob_mezczyzni",
           "liczba_osob_dzieci"]

dane = pd.read_excel(file_path, names=columns, engine='openpyxl')


dane["keso"] = dane["keso"].astype(str)
dane["keso"] = dane["keso"].str.extract(r"(\d{12})")[0].str.zfill(12)


dane["jednostka_typ"] = dane["keso"].str[0:2]
dane["wojew_kod"] = dane["keso"].str[2:4]
dane["powiat_kod"] = dane["keso"].str[4:6]
dane["woj_pow_kod"] = dane["wojew_kod"] + dane["powiat_kod"]

valid_dane = dane[
    dane["keso"].str.match(r"^\d{12}$", na=False) &
    (dane["powiat_kod"] != "00") &
    (dane["woj"].notna())
].copy()


numeric_cols = [
    "liczba_proc_nk_a", "liczba_proc_nk_c", "rodziny_wszczete",
    "liczba_osob_ogol", "liczba_osob_kobiety",
    "liczba_osob_mezczyzni", "liczba_osob_dzieci"
]

for col in numeric_cols:
    valid_dane[col] = pd.to_numeric(valid_dane[col], errors='coerce')


results = []
years = valid_dane["rok"].unique()

for year in years:
    year_data = valid_dane[valid_dane["rok"] == year].copy()


    grouped = year_data.groupby(["woj", "pow", "woj_pow_kod", "rok"])


    agg_data = grouped[numeric_cols].sum(min_count=1)

    meta_data = grouped[["keso", "wojew_kod", "powiat_kod"]].first()

    result = pd.concat([meta_data, agg_data], axis=1).reset_index()
    results.append(result)


final_result = pd.concat(results, ignore_index=True)


In [ ]:
Sprawozdanie_2014_2023_zagregowane = final_result.drop(["woj_pow_kod", "wojew_kod", "powiat_kod"],axis=1)



In [ ]:
Sprawozdanie_2014_2023_zagregowane

,woj,pow,rok,keso,liczba_proc_nk_a,liczba_proc_nk_c,rodziny_wszczete,liczba_osob_ogol,liczba_osob_kobiety,liczba_osob_mezczyzni,liczba_osob_dzieci
0,dolnośląskie,bolesławiecki,2023,300201011000,42,181.0,383.0,493.0,294.0,95.0,104.0
1,dolnośląskie,dzierżoniowski,2023,300202011000,65,132.0,291.0,402.0,216.0,26.0,160.0
2,dolnośląskie,górowski,2023,300204013000,23,56.0,125.0,273.0,211.0,23.0,39.0
3,dolnośląskie,głogowski,2023,200203000000,36,199.0,428.0,770.0,333.0,72.0,365.0
4,dolnośląskie,jaworski,2023,300205023000,12,59.0,119.0,123.0,93.0,8.0,22.0
...,...,...,...,...,...,...,...,...,...,...,...
3805,świętokrzyskie,sandomierski,2014,302609022000,8,220.0,145.0,196.0,NaN,NaN,NaN
3806,świętokrzyskie,skarżyski,2014,302610022000,32,62.0,166.0,215.0,NaN,NaN,NaN
3807,świętokrzyskie,starachowicki,2014,302611022000,4,80.0,218.0,680.0,NaN,NaN,NaN
3808,świętokrzyskie,staszowski,2014,302612012000,21,199.0,216.0,449.0,NaN,NaN,NaN


In [ ]:
Sprawozdanie_2014_2023_zagregowane[Sprawozdanie_2014_2023_zagregowane["pow"]=="chełmski"]

,woj,pow,rok,keso,liczba_proc_nk_a,liczba_proc_nk_c,rodziny_wszczete,liczba_osob_ogol,liczba_osob_kobiety,liczba_osob_mezczyzni,liczba_osob_dzieci
55,lubelskie,chełmski,2023,300603022000,18,73.0,142.0,235.0,131.0,13.0,91.0
56,lubelskie,chełmski,2023,300606082000,9,13.0,33.0,44.0,19.0,3.0,22.0
436,lubelskie,chełmski,2022,300603022000,13,77.0,144.0,221.0,133.0,17.0,71.0
437,lubelskie,chełmski,2022,300606082000,2,7.0,20.0,28.0,16.0,4.0,8.0
817,lubelskie,chełmski,2021,300603022000,113,112.0,174.0,309.0,187.0,56.0,66.0
818,lubelskie,chełmski,2021,300606082000,18,11.0,23.0,45.0,18.0,5.0,22.0
1198,lubelskie,chełmski,2020,300603022000,32,152.0,193.0,394.0,241.0,61.0,92.0
1199,lubelskie,chełmski,2020,300606082000,1,15.0,12.0,44.0,22.0,4.0,18.0
1579,lubelskie,chełmski,2019,300603022000,34,160.0,212.0,403.0,223.0,71.0,109.0
1580,lubelskie,chełmski,2019,300606082000,1,16.0,20.0,63.0,26.0,4.0,33.0


In [ ]:
Sprawozdanie_2014_2023_zagregowane.to_excel("/content/drive/MyDrive/praca_licencjacka/Sprawozdanie_2014_2023_zagregowane.xlsx", sheet_name= "Sprawozdanie_2014_2023_zagregowane" ,index=False)

/usr/local/lib/python3.11/dist-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


In [ ]:
with pd.ExcelWriter('Sprawozdanie_2014_2023_zagregowane.xlsx') as writer:
 Sprawozdanie_2014_2023_zagregowane.to_excel(writer, index=False, sheet_name='Sprawozdanie_2014_2023_zagregowane')

In [ ]:
Sprawozdanie_2014_2023_zagregowane.to_csv('Sprawozdanie_2014_2023_zagregowane.csv', index=False)
